In [98]:
from sqlalchemy import create_engine, Column, Integer, String, Time, ForeignKey, UniqueConstraint
from sqlalchemy.orm import sessionmaker, DeclarativeBase, relationship
import pandas as pd
import numpy as np

df_samples_columns = pd.read_pickle('../data/samples.pkl').columns
df_samples_head = pd.read_pickle('../data/samples.pkl').head()
df_samples = pd.read_pickle('../data/samples.pkl')

df_bands = pd.read_pickle('../data/samples.pkl').loc[:, ['band', 'band_note']].drop_duplicates().head().replace({np.nan: None})
bands_list = list(df_bands.itertuples(index=False, name=None))
bands_list

[('Age of Chance', None),
 ('Amgod', None),
 ('Aslan Faction', None),
 ('ATD Convention', None),
 ('Bassomatic', None)]

In [42]:
print(df_samples_columns)
df_albums = pd.read_pickle('../data/samples.pkl').loc[:, ['band', 'album']].drop_duplicates().head().replace({np.nan: None})
albums_list = list(df_albums.itertuples(index=False, name=None))
albums_list

Index(['sample_note', 'timestamps', 'sample', 'album', 'song', 'band',
       'groups', 'songs', 'samples', 'points', 'name', 'order', 'band_note'],
      dtype='object')


[('Age of Chance', 'One Thousand Years of Trouble'),
 ('Amgod', 'Half Rotten and Decayed'),
 ('Aslan Faction', 'Blunt Force Trauma'),
 ('ATD Convention', '(Demo)'),
 ('Bassomatic', 'Science & Melody')]

In [44]:
df_albums = pd.read_pickle('../data/samples.pkl').loc[:, ['band', 'album', 'band_note']].drop_duplicates().head().replace({np.nan: None})
albums_list = list(df_albums.itertuples(index=False, name=None))
albums_list

[('Age of Chance', 'One Thousand Years of Trouble', None),
 ('Amgod', 'Half Rotten and Decayed', None),
 ('Aslan Faction', 'Blunt Force Trauma', None),
 ('ATD Convention', '(Demo)', None),
 ('Bassomatic', 'Science & Melody', None)]

In [120]:
class Base(DeclarativeBase):
    pass

class BandInfo(Base):
    __tablename__ = 'band_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, unique=True)
    note = Column(String, nullable=True)
    albums = relationship("AlbumInfo", back_populates="band")
    
    def __str__(self):
        return f"name: {self.name}, has_note: {bool(self.note)}"

class AlbumInfo(Base):
    __tablename__ = 'album_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    band_id = Column(Integer, ForeignKey(BandInfo.id))
    band = relationship("BandInfo", back_populates="albums")
    songs = relationship("SongInfo", back_populates="album")

    __table_args__ = (
        UniqueConstraint('name', 'band_id', name='unique_album'),
    )

    def __str__(self):
        return f"name: {self.name}, band: {self.band}"

class SongInfo(Base):
    __tablename__ = 'song_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    album_id = Column(Integer, ForeignKey(AlbumInfo.id))
    album = relationship("AlbumInfo", back_populates="songs")
    samples = relationship("SampleInfo", back_populates="song")

    __table_args__ = (
        UniqueConstraint('name', 'album_id', name='unique_song'),
    )

    def __str__(self):
        return f"name: {self.name}, album: {self.album}"

class SourceInfo(Base):
    __tablename__ = 'source_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    timestamp_source = Column(Time, nullable=True)
    samples = relationship("SampleInfo", back_populates="source")

class SampleInfo(Base):
    __tablename__ = 'sample_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(String)
    song_id = Column(Integer, ForeignKey(SongInfo.id))
    song = relationship("SongInfo", back_populates="samples")
    source_id = Column(Integer, ForeignKey(SourceInfo.id))
    source = relationship("SourceInfo", back_populates="samples")
    timestamp_song = Column(Time, nullable=True)
    note = Column(String)

engine = create_engine("sqlite:///samples.db", echo=False)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()


In [103]:
display(df_samples_head)
['sample_note', 'timestamps', 'sample']
['song', 'name']
df_samples_columns

,sample_note,timestamps,sample,album,song,band,groups,songs,samples,points,name,order,band_note
0,Note: Police robot addressing crowd gathering ...,None,"Move on, move on.",One Thousand Years of Trouble,This is Crush Collision,Age of Chance,87,116,221,738,Blade Runner,1,NaN
1,"Note: ""All those moments will be lost in time""",['0:12'],All diese Momente werden verloren sein in der ...,Half Rotten and Decayed,Silence besides the Sun,Amgod,87,116,221,738,Blade Runner,1,NaN
2,"Note: ""Time to die.""",['0:24'],Zeit zu sterben.,Half Rotten and Decayed,Silence besides the Sun,Amgod,87,116,221,738,Blade Runner,1,NaN
3,Note: Just before the final fight with Deckard,['0:48'],Roy howling like a wolf,Half Rotten and Decayed,Silence besides the Sun,Amgod,87,116,221,738,Blade Runner,1,NaN
4,"Note: ""Attack ships on fire off the shoulder o...",['1:36'],"Gigantische Schiffe, die brannten, draußen vor...",Half Rotten and Decayed,Silence besides the Sun,Amgod,87,116,221,738,Blade Runner,1,NaN


Index(['sample_note', 'timestamps', 'sample', 'album', 'song', 'band',
       'groups', 'songs', 'samples', 'points', 'name', 'order', 'band_note'],
      dtype='object')

In [121]:

stored = {
    'bands': {},
    'albums': {},
    'songs': {},
    'sample_sources': {},
    'samples': {},
}
for _, row in df_samples.iterrows():
    bandname = row['band']
    if bandname not in stored['bands']:
        band = BandInfo(name=bandname, note=row['band_note'])
        session.add(band)
        session.commit()
        stored['bands'][bandname] = band
    else:
        band = stored['bands'][bandname]
    # print(band)
    
    albumname = row['album']
    if albumname not in stored['albums']:
        album = AlbumInfo(name=albumname, band=band)
        session.add(album)
        session.commit()
        stored['albums'][albumname] = album
    else:
        album = stored['albums'][albumname]
    # print(album)

    songname = row['song']
    if songname not in stored['songs']:
        song = SongInfo(name=songname, album=album)
        session.add(song)
        session.commit()
        stored['songs'][songname] = song
    else:
        song = stored['songs'][songname]
    # print(song)

    sourcename = row['name']
    if sourcename not in stored['sample_sources']:
        source = SourceInfo(name=sourcename)
        session.add(source)
        session.commit()
        stored['sample_sources'][sourcename] = source
    else:
        source = stored['sample_sources'][sourcename]
    # print(song)

    sample = row['sample']
    if sample not in stored['samples']:
        sample = SampleInfo(text=sample, song=song, source=source)
        session.add(sample)
        session.commit()
        stored['samples'][sample] = sample
    else:
        sample = stored['samples'][sample]
    # print(song)

